In [1]:
import paddle.fluid as fluid
import paddle
import matplotlib.pyplot as plt
import numpy as np
from paddle.fluid.debugger import draw_block_graphviz

In [2]:
class NumpyInitializer(fluid.initializer.Initializer):
    
    def __init__(self, ndarray):
        super(NumpyInitializer, self).__init__()
        self._ndarray = ndarray.astype('float32')

    def __call__(self, var, block):
        values = [float(v) for v in self._ndarray.flat]
        op = block.append_op(
            type="assign_value",
            outputs={"Out": [var]},
            attrs={
                "shape": var.shape,
                "dtype": int(var.dtype),
                "fp32_values": values
            })
        var.op = op
        return op

In [3]:
w = np.arange(10).reshape(5, 2).astype('float32')
b = np.arange(2).astype('float32')

x = fluid.layers.data('x', [-1, 5], dtype='float32')
y = fluid.layers.fc(x, size=2, param_attr=NumpyInitializer(w), bias_attr=NumpyInitializer(b))

In [4]:
draw_block_graphviz(y.block, path="g.pdf")

In [5]:
exe = fluid.executor.Executor(fluid.CPUPlace())
exe.run(fluid.default_startup_program())

x_input = np.ones((1, 5)).astype('float32')
exe.run(fetch_list=[y], feed={'x': x_input}) # [20, 26.]

[array([[20., 26.]], dtype=float32)]

In [6]:
fluid.default_startup_program()

blocks {
  idx: 0
  parent_idx: -1
  vars {
    name: "fc_0.b_0"
    type {
      type: LOD_TENSOR
      lod_tensor {
        tensor {
          data_type: FP32
          dims: 2
        }
      }
    }
    persistable: true
  }
  vars {
    name: "fc_0.w_0"
    type {
      type: LOD_TENSOR
      lod_tensor {
        tensor {
          data_type: FP32
          dims: 5
          dims: 2
        }
      }
    }
    persistable: true
  }
  ops {
    outputs {
      parameter: "Out"
      arguments: "fc_0.w_0"
    }
    type: "assign_value"
    attrs {
      name: "op_role_var"
      type: STRINGS
    }
    attrs {
      name: "int32_values"
      type: INTS
    }
    attrs {
      name: "op_role"
      type: INT
      i: 0
    }
    attrs {
      name: "fp32_values"
      type: FLOATS
      floats: 0.0
      floats: 1.0
      floats: 2.0
      floats: 3.0
      floats: 4.0
      floats: 5.0
      floats: 6.0
      floats: 7.0
      floats: 8.0
      floats: 9.0
    }
    attrs {
      n